### 一、项目环境搭建（请先挂载 Google Drive）

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 指定 clone 多说话人的代码分支：
!git clone -b bert_vits_aishell3 https://github.com/luuumity/vits_chinese.git

# 安装相关依赖：
%cd vits_chinese
!pip install -r requirements.txt

# set up MAS对齐
%cd monotonic_align
!mkdir monotonic_align
# colab会隐藏生成的 .so 文件！用 ls 命令可以查看到
!python setup.py build_ext --inplace
!ls monotonic_align
# 也可以直接复制之前本地build出来的文件：(记得先挂载Google Drive)
# !cp "../../drive/MyDrive/core.cpython-38-darwin.so" ./monotonic_align/
%cd ..

# 下载韵律模型：
%cd bert
!wget https://github.com/PlayVoice/vits_chinese/releases/download/v1.0/prosody_model.pt
%cd ..

# 下载多发言人预训练模型：
!wget https://github.com/PlayVoice/vits_chinese/releases/download/v4.0/AISHELL3_G.pth
!wget https://github.com/PlayVoice/vits_chinese/releases/download/v4.0/AISHELL3_D.pth

In [ ]:
# 多说话人底模测试
!python vits_infer.py -c configs/bert_vits.json -m AISHELL3_G.pth -i 0

DEBUG:jax._src.path:etils.epath found. Using etils.epath for file I/O.
/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [ ]:
# 多说话人底模测试（无BERT参与）
!python vits_infer_no_bert.py --config configs/bert_vits.json --model AISHELL3_G.pth -i 0

DEBUG:jax._src.path:etils.epath found. Using etils.epath for file I/O.
/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


### 二、数据预处理

In [ ]:
# 如果想要更频繁地输出信息和存档，配置文件里的 log interval 和 eval interval 可以改得更小。

In [ ]:
# 0.1 将自己的微调数据集放到 data/SJY/ 中:
!mkdir data
# 加 -r 参数才能递归地复制目录及其下文件，与 rm -r 是同理的。
!cp -r ../drive/MyDrive/SJY ./data/SJY

# 0.2 将文案文件 input.txt 移动到项目目录下：
!mv ./data/SJY/input.txt ./

# 0.3 手动创建 vits_data/ 目录:
!mkdir vits_data

In [ ]:
# 1. 音频重采样
# --wav 要写角色的父文件夹，这是由代码决定的。
# waves-16k目录会自动创建。
!python prep_resample.py --wav data/ --out vits_data/waves-16k
# 采样率检验（我自己的声音）：
!file vits_data/waves-16k/SJY/SJY001.wav

normalize waves: 100% 40/40 [00:09<00:00,  4.20it/s]
vits_data/waves-16k/SJY/SJY001.wav: RIFF (little-endian) data, WAVE audio, Microsoft PCM, 16 bit, mono 16000 Hz


In [ ]:
# 2. 拼音标注及格式规范化
# 原始文本请放在input.txt，默认输出到vits_data/labels.txt。
!python prep_pinyin.py
# 建议检查一下，如果有标注不准确可以手动更正。

拼音已成功输出到文件。


In [ ]:
# 3. 使用bert预处理
# 手动指定测试集有多少条（val_len）：
!python prep_bert.py --conf configs/bert_vits.json --data vits_data/
# 请检查 vits_data/ 下的所有内容，以及 filelists/ 里的内容。

In [ ]:
# 4. 将train用的数据，自我复制到800条以上：（加大每epoch中的数据数量，避免频繁读写）
mydata = []
temp = []
with open('./filelists/train.txt', 'r') as f:
  line = f.readline()
  while line and line != '':
    temp.append(line.strip())
    line = f.readline()

while len(mydata) <= 800:
  mydata += temp

print(f"现在有{len(mydata)}条记录")

with open('./filelists/train.txt', 'w') as f:
  for item in mydata:
    f.write(item+'\n')

现在有832条记录


In [ ]:
# 5. 数据调试
!python prep_debug.py

100% 40/40 [00:00<00:00, 1170.47it/s]


### 三、启动训练（输出默认都在logs/路径下）

本项目无法使用 Duration Predictor（dp），因为AISHELL3预训练模型用的就是 Scholastic Duration Predictor（sdp）。
\
如果微调时想用dp，加载底模型权重时就会报错。

In [ ]:
# 输出默认都在logs/路径下，记得手动下载保存，下次还可以当底模型接着用。
# 需手动开启 Tensorboard：
# 如有bug，请仿照vits-fast-fine-tuning做。
%load_ext tensorboard
%tensorboard --logdir "./logs"
!python train.py -c configs/bert_vits.json -m bert_vits

### 四、储存模型到 Google Drive

In [ ]:
!cp logs/bert_vits/G_1200.pth ../drive/MyDrive/微调方式1（无辅助数据）/test3（40条）/G_1200.pth

In [ ]:
!cp logs/bert_vits/D_1200.pth ../drive/MyDrive/微调方式1（无辅助数据）/test3（40条）/D_1200.pth

### 五、推理测试

In [ ]:
# 测试网盘中存储的模型：
!cp ../drive/MyDrive/微调方式1（无辅助数据）/test3（40条）/G_1200.pth ./G_1200.pth
!python vits_infer.py -c configs/bert_vits.json -m G_1200.pth -i 0
# 可以控制length_scale、noise_scale_w、noise_scale参数，这是所使用的 Stochastic Duration Predictor (sdp)的作用。
# !python vits_infer.py -c configs/bert_vits.json -m G_1000.pth -i 0 -l 1.1 -n 0.56

DEBUG:jax._src.path:etils.epath found. Using etils.epath for file I/O.
/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [ ]:
# 直接测试colab文件系统内的模型：
!python vits_infer.py -c configs/bert_vits.json -m logs/bert_vits/G_1200.pth -i 0

In [ ]:
# 无BERT的推理测试：
!python vits_infer_no_bert.py -c configs/bert_vits.json -m G_1200.pth -i 0

#### 本notebook最后修改日期：2024.5.2